# Modal Results from `OP2`

In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px  # used for quick test plots

from colorsys import rgb_to_hsv, hsv_to_rgb
from plotly.colors import qualitative as colorways
from plotly.subplots import make_subplots
from pyNastran.op2.op2 import read_op2, OP2

## Load `OP2` File

In [3]:
%%capture
op2 = OP2(mode='nx')
op2.read_op2('./modes.op2', build_dataframe=False)

Check data.

In [4]:
print(op2.get_op2_stats())

params:
  AUTOSPC = 'YES'
  K6ROT = 100.0
  MAXRATIO = 1000000000.0
  OGEOM = 'NO'
  POST = -1
  PRGPST = 'NO'
  RESVEC = 'YES'
  WTMASS = 0.002589999930933118
op2_results.eqexin: EQEXIN(nid, ndof, doftype); nnodes=27022
eigenvectors[1]
  isubcase = 1
  type=RealEigenvectorArray ntimes=50 nnodes=27022, table_name=OUGV1
  data: [t1, t2, t3, r1, r2, r3] shape=[50, 27022, 6] dtype=float32
  node_gridtype.shape = (27022, 2)
  sort1
  modes = [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48
 49 50]
  eigns = [  84195.1875     86258.3125    138343.578125  159460.484375
  441578.375     465481.25      508520.375     527238.5625
  666105.25      812485.        855163.1875    874208.6875
  895739.1875   1090621.5      1194446.25     1775070.375
 1781796.5      1828455.625    1844817.5      1897179.625
 1938493.       1985491.75     2128450.       2512902.75
 2626421.       3343636.75     4307535.   

Reduce sparse matrix results to usable `DataFrame`.

In [5]:
def modal_matrix_to_dataframe(op2, matrix_key):
    """Get a modal matrix indexed by frequency as a dataframe.

    Parameters
    ----------
    op2 : pyNastran.op2.op2.OP2
        OP2 object
    matrix_key : {'EFMFACS', 'MPFACS', 'MEFMASS', 'MEFWTS'}
        key of matrix to return. Assumes SORT1.

    Returns
    -------
    pandas.DataFrame
        Matrix as a DataFrame
    """

    # build dataframes if not already built
    op2.matrices[matrix_key].build_dataframe()
    op2.eigenvectors[1].build_dataframe()

    # create data frame from sparse matrix
    df = pd.DataFrame(
        data=op2.matrices[matrix_key].data.todense().T,
        columns=op2.eigenvectors[1].headers,
        index=op2.eigenvectors[1].data_frame.columns.get_level_values('Freq')
    )

    # add mode numbers as indicies
    df['mode'] = op2.eigenvectors[1].modes
    df = df.reset_index().set_index('mode', drop=True)
    df.columns = [i.capitalize() for i in df.columns]

    return df

In [6]:
efmfacs = modal_matrix_to_dataframe(op2, 'EFMFACS')
efmfacs = np.round(efmfacs,2)

Isolate critical modes contributing a mass fraction of 1% or greater.

In [17]:
crit = efmfacs[efmfacs >= 0.01].dropna(
    how='all',
    subset=['T1', 'T2', 'T3', 'R1', 'R2', 'R3']
)

# create total row 
s = efmfacs.sum()
s['Freq'] = efmfacs['Freq'].max()
# s['mode'] = np.NaN
s = pd.DataFrame(s).T
s.index = ['Sum At']
s = np.round(s, 2)

# add total row to table
crit = pd.concat([crit, s])
crit.index.name = 'Mode'

crit

,Freq,T1,T2,T3,R1,R2,R3
Mode,,,,,,,
1,46.18,0.82,NaN,NaN,NaN,0.95,NaN
2,46.74,NaN,0.83,NaN,0.96,NaN,NaN
3,59.20,NaN,NaN,0.68,NaN,NaN,NaN
4,63.55,NaN,NaN,0.05,NaN,NaN,NaN
5,105.76,0.08,NaN,NaN,NaN,NaN,0.01
6,108.59,NaN,0.09,NaN,0.01,NaN,NaN
7,113.49,0.01,NaN,NaN,NaN,NaN,0.79
8,115.56,0.02,NaN,NaN,NaN,0.01,0.14
10,143.46,NaN,NaN,NaN,0.01,NaN,NaN


## Plot

Color defaults and tools.

In [ ]:
# default colorway
color = colorways.Plotly

# color saturation tool
def hex_saturation(hex, percent):
    """Change saturation of hex color string."""

    rgb = tuple(int(hex[1:][i:i+2], 16) for i in (0, 2, 4))
    rgb_norm = list(np.array(rgb) / 255)
    hsv = np.array(rgb_to_hsv(*rgb_norm))
    hsv[1] *= percent
    rgb = list(np.array(hsv_to_rgb(*hsv)) * 255)
    return '#' + ('{:02X}' * 3).format(*[int(i) for i in rgb])

Default layout.

In [ ]:
layout=go.Layout(
    height=500,
    width=1000,
    title={'text': f"<b>Modal Effective Mass Fraction</b><br>&#8805;1%",
           'x': 0.5,
           'font': {'size': 20}},
    font={'family': 'Segoe UI',
          'size': 14},
    template='plotly_white',
    xaxis={'type': 'linear',
           'title': "Frequency (Hz)",
           'showgrid': True,
           'minor': {'dtick': 'D1'},
           'range': [20, 2000],
           'showline': True,
           'mirror': True,
           'showspikes': True,
           'hoverformat': '.0f',
           'spikethickness': 1,
           'spikemode': 'across'},
    yaxis={'title': 'Fraction',
           'showline': True,
           'mirror': True,
           'scaleanchor': 'y2',
           'scaleratio': 1,
           'dtick': 0.1,
           'rangemode': 'tozero',
           'constraintoward': 'bottom',
           'tickformat': '.0%'},
    yaxis2={'title': 'Sum',
            'side': 'right',
            'scaleanchor': 'y',
            'scaleratio': 1,
            'dtick': 0.2,
            'rangemode': 'tozero',
            'constraintoward': 'bottom',
            'hoverformat': '.0%',
            'tickformat': '.0%'},
    legend={'title': {'text': 'DOF'}},
    hovermode='closest',
    plot_bgcolor='rgb(250,250,250)',
    paper_bgcolor='rgb(250,250,250)'
)

Plot results.

In [15]:
# plot results
fig = make_subplots(specs=[[{'secondary_y': True}]])
fig.update_layout(layout)

headers = efmfacs.drop('Freq', axis=1).columns

cidx = 0
for col in headers:
    fig.add_bar(
        x=efmfacs['Freq'],
        y=efmfacs[col],
        customdata=efmfacs.index,
        name=col[:2].upper(),
        yaxis='y',
        # width=[1/10 * (10**(np.log10(i)-1))  # for logarithmic x-axis
        #         for i in efmfacs['Freq']],
        width=3,
        marker={'color': color[cidx]},
        hovertemplate="<b>Mode %{customdata}</b><br>%{x} Hz<br>%{y}",
        legendgroup=col[:2].lower()
    )

    fig.add_scatter(
        x=efmfacs['Freq'],
        y=efmfacs[col].cumsum(),
        customdata=efmfacs.index,
        name=col[:2].upper() + " Sum",
        mode='lines',
        line={'color': hex_saturation(color[cidx], 0.5),
              'width': 1.0},
        yaxis='y2',
        hovertemplate="<b>Mode %{customdata}</b><br>%{x} Hz<br>%{y}",
        legendgroup=col[:2].lower(),
        showlegend=False,
    )

    cidx += 1

fig.write_html('modes.html', include_mathjax='cdn')
fig.show()